# Estimating Causal Effects of Actors on Movie Revenue
An example in Model Based Machine Learning.

In [1]:
# Imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

import pyro
assert(pyro.__version__ == '0.4.1')
from pyro.distributions import Bernoulli
from pyro.distributions import Delta
from pyro.distributions import Normal
from pyro.distributions import Uniform
from pyro.distributions import LogNormal
from pyro.infer import SVI
from pyro.infer import Trace_ELBO
from pyro.optim import Adam
import torch.distributions.constraints as constraints
pyro.set_rng_seed(101)

# Data loader
from box_office.box_office import data_loader

## The Problem
Lets say producers of a movie approaches you with a set of actors they'd like to cast in their movie and want you to find out 2 things: <br>
    1) How much box office revenue will this movie make with this cast of actors?<br>
    2) How much of that revenue will every actor be responsible for?

## The "Data Science" Solution

You scrape IMDB, get a list of movies that have grossed 1 million or higher (that's the least a decent movie could do), and retain actors who have appeared in at least 10 movies. Then arrange it in a data frame where every movie is a row, actors are columns, and presence or absence of actors in that movie is denoted as 1 or 0. <br>
That looks something like this:

In [2]:
# Load data from the dataframe
data_loc = "box_office/data/ohe_movies.csv"
x_train_tensors, y_train_tensors, actors, og_movies = data_loader.load_tensor_data(data_loc)

In [3]:
og_movies.head()

,title_x,Adam Sandler,Alec Baldwin,Amy Adams,Angelina Jolie,Anna Faris,Anne Hathaway,Anthony Hopkins,Anthony Mackie,Antonio Banderas,...,Sylvester Stallone,Tom Cruise,Tom Hanks,Tom Wilkinson,Tommy Lee Jones,Vince Vaughn,Will Ferrell,Will Smith,Willem Dafoe,Woody Harrelson
0,Avatar,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Pirates of the Caribbean: At World's End,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Spectre,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Dark Knight Rises,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,John Carter,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0


Now we can fit a Linear Regression to this dataset, or any Linear Model for that matter even a Neural Network. We'll go with Linear Regression for now because regression estimates directly represent a proportional relation with the outcome variable i.e we can treat them as causal effects. <br>
This seems like a good idea till we inspect what happens to the regression coefficients. <br>
Fitting a Linear Model hides confounders that affect both actors and revenues. For example, genre is a confounder. Take Action and comedy. Action movies on average make more than Comedy movies and each tend to cast a different set of actors.<br> 
When unobserved the genre produces statistical dependence between if an actor is in the movie and its revenue.
So the causal estimates for every actor are biased.

Judi Dench played M in every James Bond movie from 1995 to 2012.
Cobie Smulders plays Maria Hill in every Avenger’s movies.
These are 2 well known but not massively popular actors who appear in high grossing movies.
The regression estimates for them are biased, and will lead to an overestimate of revenue for a new movie casting them. This is because our DAG looks like this: 
![DAG2](figs/ml_dag.jpg)

## A Causal Solution

We argue that there are certain common factors that go into picking a cast for a movie and the revenue that the movie generates. In Causality Theory these are called *Confounders*. So we propose the following data generation process: where certain unknown confounders *Z* influence the set of actors *A* and the revenue *R*. This is represented as a Bayesian Network.
![DAG](figs/causal_dag.jpg)

We will now stick with this generative model and figure a way to unbias our causal regression estimates. If we somehow find a way to estimate Z, then we can include it in our Regression Model and obtain unbiased causal estimates as regression coefficients.

This is how every individual function for a variable will be.

In [4]:
def f_z(params):
    """Samples from P(Z)"""    
    z_mean0 = params['z_mean0']
    z_std0 = params['z_std0']
    z = pyro.sample("z", Normal(loc = z_mean0, scale = z_std0))
    return z

def f_x(z, params):
    """
    Samples from P(X|Z)
    
    P(X|Z) is a Bernoulli with E(X|Z) = logistic(Z * W),
    where W is a parameter (matrix).  In training the W is
    hyperparameters of the W distribution are estimated such
    that in P(X|Z), the elements of the vector of X are
    conditionally independent of one another given Z.
    """
    def sample_W():
        """
        Sample the W matrix
        
        W is a parameter of P(X|Z) that is sampled from a Normal
        with location and scale hyperparameters w_mean0 and w_std0
        """
        w_mean0 = params['w_mean0']
        w_std0 = params['w_std0']
        W = pyro.sample("W", Normal(loc = w_mean0, scale = w_std0))
        return W
    W = sample_W()
    linear_exp = torch.matmul(z, W)
    # sample x using the Bernoulli likelihood
    x = pyro.sample("x", Bernoulli(logits = linear_exp))
    return x

def f_y(x, z, params):
    """
    Samples from P(Y|X, Z)
    
    Y is sampled from a Gaussian where the mean is an
    affine combination of X and Z.  Bayesian linear
    regression is used to estimate the parameters of
    this affine transformation  function.  Use torch.nn.Module to create
    the Bayesian linear regression component of the overall
    model.
    """
    predictors = torch.cat((x, z), 1)

    w = pyro.sample('weight', Normal(params['weight_mean0'], params['weight_std0']))
    b = pyro.sample('bias', Normal(params['bias_mean0'], params['bias_std0']))

    y_hat = (w * predictors).sum(dim=1) + b
    # variance of distribution centered around y
    sigma = pyro.sample('sigma', Normal(params['sigma_mean0'], params['sigma_std0']))
    with pyro.iarange('data', len(predictors)):
        pyro.sample('y', LogNormal(y_hat, sigma))
        return y_hat

And this is our complete generative causal model.

In [5]:
def model(params):
    """The full generative causal model"""
    z = f_z(params)
    x = f_x(z, params)
    y = f_y(x, z, params)
    return {'z': z, 'x': x, 'y': y}

## How to infer Z?

If we could somehow measure all confounders that affect choice of cast and revenue generated by that cast, then we could  condition on them and obtain unbiased estimates. This is the Ignorability assumption: that the outcome is independent of treatment assignment (choice of actors), so now average difference in outcomes between two groups of actors can only be attributable to the treatment (actors). The problem here is that it's impossible to check if we've measured all confounders. So Yixin Wang and David M. Blei proposed an algorithm; “The Deconfounder” to sidestep the search for confounders because its impossible to exhaust them.
They find a latent variable model over the causes.
Use it to infer latent variables for each individual movie.
Then use this inferred variable as a “substitute confounder” and get back to treating this as a regression problem with the inferred variables as more data. So we use a probabilistic PCA model over actors to infer the latent variables that explain the distribution of actors.

In [6]:
def step_1_guide(params):
    """
    Guide function for fitting P(Z) and P(X|Z) from data
    """
    # Infer z hyperparams
    qz_mean = pyro.param("qz_mean", params['z_mean0'])
    qz_stddv = pyro.param("qz_stddv", params['z_std0'],
                         constraint=constraints.positive)
    
    z = pyro.sample("z", Normal(loc = qz_mean, scale = qz_stddv))
    
    # Infer w params
    qw_mean = pyro.param("qw_mean", params["w_mean0"])
    qw_stddv = pyro.param("qw_stddv", params["w_std0"],
                          constraint=constraints.positive)
    w = pyro.sample("w", Normal(loc = qw_mean, scale = qw_stddv))

We use Pyro's _"guide"_ functionality to infer $P(Z)$ and $P(X|Z)$ using Stochastic Variational Inference, a scalable algorithm for approximating posterior distributions. For this, we define the above guide function.

The primary goal however, is to estimate the causal estimates of actors which are the linear regression coefficients for each actor. For this we will write another guide function: one that optimizes for the linear regression parameters.

In [7]:
def step_2_guide(params):
    # Z and W are just sampled using param values optimized in previous step
    z = pyro.sample("z", Normal(loc = params['qz_mean'], scale = params['qz_stddv']))
    w = pyro.sample("w", Normal(loc = params['qw_mean'], scale = params['qw_stddv']))
    
    # Infer regression params
    # parameters of (w : weight)
    w_loc = pyro.param('w_loc', params['weight_mean0'])
    w_scale = pyro.param('w_scale', params['weight_std0'])

    # parameters of (b : bias)
    b_loc = pyro.param('b_loc', params['bias_mean0'])
    b_scale = pyro.param('b_scale', params['bias_std0'])
    # parameters of (sigma)
    sigma_loc = pyro.param('sigma_loc', params['sigma_mean0'])
    sigma_scale = pyro.param('sigma_scale', params['sigma_std0'])

    # sample (w, b, sigma)
    w = pyro.sample('weight', Normal(w_loc, w_scale))
    b = pyro.sample('bias', Normal(b_loc, b_scale))
    sigma = pyro.sample('sigma', Normal(sigma_loc, sigma_scale))

The primary difference, between what Wang and Blei have done, and what we will do here, is that we have implemented our beliefs as a generative model. The factor model (Probabilistic PCA) and the regression have been combined into one model over the DAG. <br>
It's important to understand that Wang et. al. separate the estimation of 𝑍 from the estimation of regression parameters.
This is done because 𝑍 by construction renders all causes (actors) independent of each other. 
Including the outcome (revenue) while learning parameters of 𝑍 would make the revenue conditionally independent of the actors which violates our primary assumption that actors are a cause of movie revenue.
So they estimate 𝑍, then hardcode it into their regression problem.<br>
<br>
We handle this by running a 2 step training process in Pyro. That is with two different guide functions over the same DAG to optimize different parameters conditional on certain variables at a time. <br>
The first learns posterior of 𝑍 and 𝑊(a parameter of P(X|Z)) conditional on 𝑋. <br>
The second learns the regression parameters conditional on what we what we know about 𝑋, what we just learnt about 𝑊, and what we know about 𝑌. <br>
Once they are defined, we only need to train this generative model.

In [8]:
def training_step_1(x_data, params):
    
    adam_params = {"lr": 0.0005}
    optimizer = Adam(adam_params)

    conditioned_on_x = pyro.condition(model, data = {"x" : x_data})
    svi = SVI(conditioned_on_x, step_1_guide, optimizer, loss=Trace_ELBO())
    
    print("\n Training Z marginal and W parameter marginal...")

    n_steps = 2000
    pyro.set_rng_seed(101)
    # do gradient steps
    pyro.get_param_store().clear()
    for step in range(n_steps):
        loss = svi.step(params)
        if step % 100 == 0:
            print("[iteration %04d] loss: %.4f" % (step + 1, loss/len(x_data)))
            
    # grab the learned variational parameters
    
    updated_params = {k: v for k, v in params.items()}
    for name, value in pyro.get_param_store().items():
        print("Updating value of hypermeter{}".format(name))
        updated_params[name] = value.detach()
        
    return updated_params

In [9]:
def training_step_2(x_data, y_data, params):
    print("Training Bayesian regression parameters...")
    pyro.set_rng_seed(101)
    num_iterations = 1500
    pyro.clear_param_store()
    # Create a regression model
    optim = Adam({"lr": 0.003})
    conditioned_on_x_and_y = pyro.condition(model, data = {
        "x": x_data,
        "y" : y_data
    })

    svi = SVI(conditioned_on_x_and_y, step_2_guide, optim, loss=Trace_ELBO(), num_samples=1000)
    for step in range(num_iterations):
        loss = svi.step(params)
        if step % 100 == 0:
            print("[iteration %04d] loss: %.4f" % (step + 1, loss/len(x_data)))
    
    
    updated_params = {k: v for k, v in params.items()}
    for name, value in pyro.get_param_store().items():
        print("Updating value of hypermeter: {}".format(name))
        updated_params[name] = value.detach()
    print("Training complete.")
    return updated_params

In [10]:
def train_model():
    num_datapoints, data_dim = x_train_tensors.shape
    
    latent_dim = 30 # can be changed
    params0 = {
        'z_mean0': torch.zeros([num_datapoints, latent_dim]),
        'z_std0' : torch.ones([num_datapoints, latent_dim]),
        'w_mean0' : torch.zeros([latent_dim, data_dim]),
        'w_std0' : torch.ones([latent_dim, data_dim]),
        'weight_mean0': torch.zeros(data_dim + latent_dim),
        'weight_std0': torch.ones(data_dim + latent_dim),
        'bias_mean0': torch.tensor(0.),
        'bias_std0': torch.tensor(1.),
        'sigma_mean0' : torch.tensor(1.),
        'sigma_std0' : torch.tensor(0.05)
    } # These are our priors

    params1 = training_step_1(x_train_tensors, params0)
    params2 = training_step_2(x_train_tensors, y_train_tensors, params1)
    return params1, params2

And now, we train the model to infer latent variable distributions and Bayesian Regression coefficients.

In [11]:
p1, p2 = train_model()


 Training Z marginal and W parameter marginal...
[iteration 0001] loss: 304.3461
[iteration 0101] loss: 294.8547
[iteration 0201] loss: 290.4372
[iteration 0301] loss: 281.5974
[iteration 0401] loss: 274.5142
[iteration 0501] loss: 273.1243
[iteration 0601] loss: 261.1506
[iteration 0701] loss: 260.4133
[iteration 0801] loss: 250.2013
[iteration 0901] loss: 248.1334
[iteration 1001] loss: 250.6025
[iteration 1101] loss: 246.0507
[iteration 1201] loss: 240.1931
[iteration 1301] loss: 232.8412
[iteration 1401] loss: 229.0232
[iteration 1501] loss: 215.9541
[iteration 1601] loss: 209.8044
[iteration 1701] loss: 201.3092
[iteration 1801] loss: 187.6613
[iteration 1901] loss: 183.0304
Updating value of hypermeterqz_mean
Updating value of hypermeterqz_stddv
Updating value of hypermeterqw_mean
Updating value of hypermeterqw_stddv
Training Bayesian regression parameters...
[iteration 0001] loss: 258.9689


C:\ProgramData\Anaconda3\lib\site-packages\pyro\infer\trace_elbo.py:135: UserWarning: Encountered NaN: loss
  warn_if_nan(loss, "loss")


[iteration 0101] loss: 263.9321
[iteration 0201] loss: 222.0467
[iteration 0301] loss: 206.6688
[iteration 0401] loss: 199.0078
[iteration 0501] loss: 194.0477
[iteration 0601] loss: 178.9697
[iteration 0701] loss: 181.7087
[iteration 0801] loss: 179.2907
[iteration 0901] loss: 178.9970
[iteration 1001] loss: 167.8895
[iteration 1101] loss: 175.2319
[iteration 1201] loss: 173.3212
[iteration 1301] loss: 171.9837
[iteration 1401] loss: 170.9341
Updating value of hypermeter: w_loc
Updating value of hypermeter: w_scale
Updating value of hypermeter: b_loc
Updating value of hypermeter: b_scale
Updating value of hypermeter: sigma_loc
Updating value of hypermeter: sigma_scale
Training complete.


### Causal effect of actors with and without confounding
Because we have implemented all our assumptions as one generative model, finding causal estimates of actors is as simple as calling the condition and do queries from Pyro.
Causal effect of actors without confounding: $E[Y|X=1] - E[Y|X=0]$ <br>
Causal effect of actors without confounding: $E[Y|do(X=1)] - E[Y|do(X=0)]$

In [12]:
def condCausal(their_tensors, absent_tensors, movie_inds):
    their_cond = pyro.condition(model, data = {"x" : their_tensors})
    absent_cond = pyro.condition(model, data = {"x" : absent_tensors})
    
    their_y = []
    for _ in range(1000):
        their_y.append(torch.sum(their_cond(p2)['y'][movie_inds]).item())
    
    absent_y = []
    for _ in range(1000):
        absent_y.append(torch.sum(absent_cond(p2)['y'][movie_inds]).item())
    
    their_mean = np.mean(their_y)
    absent_mean = np.mean(absent_y)
    causal_effect_noconf = their_mean - absent_mean
    
    return causal_effect_noconf

In [13]:
def doCausal(their_tensors, absent_tensors, movie_inds):
    # With confounding
    their_do = pyro.do(model, data = {"x" : their_tensors})
    absent_do = pyro.do(model, data = {"x" : absent_tensors})
    
    their_do_y = []
    for _ in range(1000):
        their_do_y.append(torch.sum(their_do(p2)['y'][movie_inds]).item())
    
    absent_do_y = []
    for _ in range(1000):
        absent_do_y.append(torch.sum(absent_do(p2)['y'][movie_inds]).item())
    
    their_do_mean = np.mean(their_do_y)
    absent_do_mean = np.mean(absent_do_y)
    causal_effect_conf = their_do_mean - absent_do_mean
    
    return causal_effect_conf

In [14]:
def causal_effects(actor):
    # Get all movies where that actor is present
    # Make him/her absent, and then get conditional expectation
    
    actor_tensor = pd.DataFrame(x_train_tensors.numpy(), columns=actors[1:])
    # All movies where actor is present
    movie_inds = actor_tensor.index[actor_tensor[actor] == 1.0]

    absent_movies = actor_tensor.copy()
    absent_movies[actor] = 0
    
    their_tensors = x_train_tensors
    absent_tensors = torch.tensor(absent_movies.to_numpy(dtype = 'float32'))
    
    cond_effect_mean = condCausal(their_tensors, absent_tensors, movie_inds)
    do_effect_mean = doCausal(their_tensors, absent_tensors, movie_inds)
#     print(their_tensors.shape, absent_tensors.shape)
    diff_mean = cond_effect_mean - do_effect_mean
    if diff_mean > 0:
        status = "Overvalued"
    else: status = "Undervalued"
    
    print("Causal conditional effect: ", cond_effect_mean)
    
    print("Causal Do effect: ", do_effect_mean)
    
    print("Diff: ", diff_mean)
    
    print("Status: ", status)

We can now call these queries on the actors for whome we'd like to see biased and unbiased estimates for. Cobie Smulders and Judi Dench are the examples we set out to prove our point with, and the our generative model does obtain debiased estimates of their causal effect on revenue. 

In [15]:
causal_effects("Cobie Smulders")

Causal conditional effect:  21.012677001953136
Causal Do effect:  19.785036128997803
Diff:  1.2276408729553339
Status:  Overvalued


In [16]:
causal_effects("Judi Dench")

Causal conditional effect:  77.29178205871585
Causal Do effect:  76.0234787902832
Diff:  1.2683032684326463
Status:  Overvalued


Causal effect of Actor $j$ on Revenue without conditioning for the confounders can be obtained by $E[Y|X_j=1]−E[Y|X_j=0]$
This retains the original DAG and samples at site $Y$.

Causal effect of Actor j on Revenue without conditioning for the confounders can be obtained by $E[Y|do(X_j=1)] - E[Y|do(X_j=0)]$
This mutates the DAG as the *do* operator removes all incoming edges into a node. We then sample at site $Y$.